# 需要自己在Github上面成立想要的回答問題方式，包括訓練集與測試集


# GenAI HW6: LLM Values Alignment
## Objectives
- Learn how to align a model's behavior using labelled preference data.

If you have any questions, please contact the TAs via TA hours, NTU COOL, or email to ntu-gen-ai-2024-spring-ta@googlegroups.com

## Install and import necessary libraries  (~2 min)
### Ignore the warning if the blockes finish successfully.

In [1]:
!pip install -q bitsandbytes==0.43.1 datasets==2.19.0 peft==0.10.0 trl==0.8.6 accelerate==0.29.3

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 18.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.1/199.1 kB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.2/245.2 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.6/297.6 kB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.0/172.0 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.6/104.6 kB 266.9 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 13.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.6.1 re

In [2]:
import os
import torch
import re
import json
import gdown
from datasets import Dataset
import pandas as pd
from peft import LoraConfig
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, BitsAndBytesConfig, GenerationConfig
from tqdm.auto import tqdm
from trl import DPOTrainer

## Load dataset

In [3]:
!git clone https://github.com/ChiangYen-Wen/AI_Test

Cloning into 'AI_Test'...
remote: Enumerating objects: 10, done.
remote: Counting objects: 100% (10/10), done.
remote: Compressing objects: 100% (10/10), done.
remote: Total 10 (delta 1), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (10/10), 8.49 KiB | 8.49 MiB/s, done.
Resolving deltas: 100% (1/1), done.


In [4]:
# Open and load the json dataset
with open("/content/AI_Test/modified_labelled_data.json", 'r') as jsonfile:
    full_data = json.load(jsonfile)

with open("/content/AI_Test/modified_test_prompt.json", 'r') as jsonfile:
    test_data = json.load(jsonfile)

## Load model

In [5]:
model = AutoModelForCausalLM.from_pretrained(
    'MediaTek-Research/Breeze-7B-Instruct-v0_1',
    device_map='auto',
    trust_remote_code=True,
    quantization_config=BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_compute_dtype=torch.bfloat16,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type='nf4'
    )
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/618 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.99G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

## Get response from the original model

In [6]:
tokenizer = AutoTokenizer.from_pretrained('MediaTek-Research/Breeze-7B-Instruct-v0_1')
tokenizer.padding_side = "right"
tokenizer.pad_token = tokenizer.eos_token

def data_formulate(data):
    messages = [
        {"role": "system", "content": '回覆請少於20字'},
        {"role": "user", "content": data['prompt']},
    ]
    prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    return prompt

original_model_response = []
for data in tqdm(test_data):
    id = data['id']
    print(f'Question {id}:\n'+data['prompt'])
    inputs = tokenizer(data_formulate(data), return_tensors="pt").to('cuda')
    generation_config=GenerationConfig(
            do_sample=False,
            max_new_tokens = 200,
            pad_token_id = tokenizer.pad_token_id
    )
    output = model.generate(**inputs, generation_config=generation_config)
    output = tokenizer.batch_decode(output, skip_special_tokens=True)[0].split('[/INST] ')[1]
    original_model_response.append(output)
    print('Response from original model:\n'+output+'\n')

tokenizer_config.json:   0%|          | 0.00/2.29k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/911k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.79M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/39.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


  0%|          | 0/10 [00:00<?, ?it/s]

Question 1:
台灣股市在未來五年是否能持續成長？
Response from original model:
台灣股市在未來五年可能會持續成長，但可能存在不確定性。經濟成長、企業表現、國際情勢等因素影響。投資者應審慎評估風險與機會。

Question 2:
台灣股市是否能應對全球經濟變動的挑戰？
Response from original model:
台灣股市在面對全球經濟變動的挑戰上，具有一定程度的韌性和韌性。然而，其表現仍受制於國際市場波動、貿易戰影響、以及國內政策調整等因素。投資人應謹慎評估風險並採取適當的投資策略。

Question 3:
台灣股市是否會受益於科技產業的發展？
Response from original model:
是的，台灣股市可能受惠於科技產業的發展，尤其是半導體、5G、人工智慧、電動車等領域。然而，股市表現也受總體經濟、政策、國際情勢等因素影響，投資者需審慎評估風險與機會。

Question 4:
台灣股市在未來是否能吸引更多外國投資？
Response from original model:
台灣股市在未來可能會吸引更多外國投資，主要因全球投資趨勢轉向亞洲、台灣經濟穩定成長、科技產業蓬勃發展、以及政府推動金融自由化。然而，投資環境的不透明度、法規調整速度緩慢、以及地緣政治風險等因素仍需解決。

Question 5:
台灣股市的長期投資潛力是否值得信賴？
Response from original model:
台灣股市的長期投資潛力值得信賴，因台灣經濟持續成長、科技業蓬勃發、以及政府推動創新政策，提供投資者長期報酬率的機會。然投資仍應注意市場風險、多元布局及長期投資策略。

Question 6:
台灣股市在未來是否能穩定成長？
Response from original model:
台灣股市在未來可能會經歷波動，但長期而言，可以期待穩定成長。台灣擁有強勁的經濟成長潛力、科技產業優勢、以及政府支持政策，有助於維持市場穩定性。然需注意全球經濟情勢、貿易關係、以及國內外政治環境等因素的影響。

Question 7:
台灣股市是否會成為亞洲金融中心的重要一環？
Response from original model:
台灣股市有潛力成為亞洲金融中心的重要一環，但需加強金融法規、投資環境、國際合作等方面的

## Set parameters
### You only need to modify this block. Please don’t alter any other parts.

In [7]:
num_epoch = 2        #模型會完全看過訓練資料集幾次，1~3
data_size = 50       #願意納入訓練資料的比數
support_ratio = 0.95    #支持比率

## Prepare training data

In [8]:
# Select part of the data for training
training_data = full_data[:data_size]

# Define the size of the support dataset
support_data_size = int(data_size * support_ratio)

# Prepare the data for the training dataset
prompt_list = [data_formulate(data) for data in training_data]
chosen_list = [data['support'] for data in training_data[:support_data_size]] + [data['oppose'] for data in training_data[support_data_size:]]
rejected_list = [data['oppose'] for data in training_data[:support_data_size]] + [data['support'] for data in training_data[support_data_size:]]
position_list = ['support' for _ in range(support_data_size)] + ['oppose' for _ in range(data_size - support_data_size)]

# Create the training dataset
train_dataset = Dataset.from_dict({'prompt': prompt_list, 'position': position_list, 'chosen': chosen_list, 'rejected': rejected_list})
pd.DataFrame(train_dataset).rename(columns={"chosen": "preferred", "rejected": "non-preferred"})

,prompt,position,preferred,non-preferred
0,<s>回覆請少於20字 [INST] 台灣股市未來五年是否能穩定增長？ [/INST],support,台灣股市受益於強勁的科技產業和穩健的經濟政策，未來五年有望穩定增長。,全球經濟的不確定性和台灣內部結構性問題可能導致股市未來五年面臨巨大挑戰。
1,<s>回覆請少於20字 [INST] 台灣股市未來是否能吸引更多外國投資？ [/INST],support,台灣股市有著強大的產業基礎和投資潛力，未來將吸引更多外資進駐。,外資可能因地緣政治風險和市場波動而對台灣股市保持觀望。
2,<s>回覆請少於20字 [INST] 科技產業是否會成為台灣股市的主要動力？ [/INST],support,台灣科技產業蓬勃發展，將成為推動股市成長的主要動力。,雖然科技產業強勢，但全球供應鏈的變動可能削弱其對股市的正面影響。
3,<s>回覆請少於20字 [INST] 台灣股市是否能在全球市場中保持競爭力？ [/INST],support,台灣的創新能力和穩定的經濟政策將幫助股市在全球市場中保持競爭力。,台灣股市可能因缺乏國際化和市場深度而在全球競爭中逐漸失去優勢。
4,<s>回覆請少於20字 [INST] 台灣股市的長期投資潛力是否值得信賴？ [/INST],support,台灣股市具備穩定的經濟基礎和成長潛力，長期投資價值顯著。,市場波動大且受多重風險影響，長期投資的安全性可能不如預期。
5,<s>回覆請少於20字 [INST] 台灣股市能否克服未來可能的經濟危機？ [/INST],support,台灣股市具備強大的韌性和應對危機的能力，能夠克服未來的挑戰。,未來的經濟危機可能會對台灣股市造成嚴重打擊，難以迅速恢復。
6,<s>回覆請少於20字 [INST] 台灣股市未來五年是否會持續成長？ [/INST],support,隨著國際經濟復甦，台灣股市在未來五年將持續成長。,全球經濟環境的不穩定可能會抑制台灣股市的成長。
7,<s>回覆請少於20字 [INST] 台灣股市的未來是否充滿機遇？ [/INST],support,台灣股市未來充滿機遇，投資者可以期待穩定的收益。,台灣股市面臨的風險和不確定性可能超過機遇。
8,<s>回覆請少於20字 [INST] 台灣股市未來是否能吸引年輕投資者？ [/INST],support,台灣股市具有創新性和成長潛力，將吸引越來越多的年輕投資者。,年輕投資者可能更偏好國際市場，而非台灣股市。
9,<s>回覆請少於20字 [INST] 台灣股市是否能成為亞洲金融中心的一部分？ [/IN...,support,台灣擁有強大的經濟基礎和金融體系，未來有望成為亞洲金融中心的一部分。,地緣政治因素和市場規模限制了台灣成為亞洲金融中心的重要性。


## Training

In [9]:
training_args = TrainingArguments(
    output_dir='./',
    per_device_train_batch_size=1,
    num_train_epochs=num_epoch,
    gradient_accumulation_steps=8,
    gradient_checkpointing=False,
    learning_rate=2e-4,
    optim="paged_adamw_8bit",
    logging_steps = 1,
    warmup_ratio = 0.1,
    report_to = 'none'
)

In [10]:
peft_config = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=64,
    bias="none",
    task_type="CAUSAL_LM",
)

In [11]:
dpo_trainer = DPOTrainer(
    model,
    args=training_args,
    beta=0.1,
    train_dataset=train_dataset,
    tokenizer=tokenizer,
    peft_config=peft_config,
)

/usr/local/lib/python3.10/dist-packages/trl/trainer/dpo_trainer.py:300: UserWarning: `max_length` is not set in the DPOTrainer's init it will default to `512` by default, but you should do it yourself in the future.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/dpo_trainer.py:307: UserWarning: `max_prompt_length` is not set in the DPOTrainer's init it will default to `128` by default, but you should do it yourself in the future.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/dpo_trainer.py:332: UserWarning: When using DPODataCollatorWithPadding, you should set `remove_unused_columns=False` in your TrainingArguments we have set it for you, but you should do it yourself in the future.
  warnings.warn(


Map:   0%|          | 0/50 [00:00<?, ? examples/s]

In [12]:
dpo_trainer.train()

Could not estimate the number of tokens of the input, floating-point operations will not be computed


Step,Training Loss
1,0.693100
2,0.693100
3,0.557600
4,0.505800
5,0.524100
6,0.338100
7,0.188700
8,0.236200
9,0.063000
10,0.027300


TrainOutput(global_step=12, training_loss=0.3415472889319062, metrics={'train_runtime': 336.746, 'train_samples_per_second': 0.297, 'train_steps_per_second': 0.036, 'total_flos': 0.0, 'train_loss': 0.3415472889319062, 'epoch': 1.92})

## Get response from the trained model

In [13]:
trained_model_response = []
for data in tqdm(test_data):
    id = data['id']
    print(f'Question {id}:\n'+data['prompt'])
    inputs = tokenizer(data_formulate(data), return_tensors="pt").to('cuda')
    generation_config=GenerationConfig(
            do_sample=False,
            max_new_tokens = 200,
            pad_token_id = tokenizer.pad_token_id
    )
    output = model.generate(**inputs, generation_config=generation_config)
    output = tokenizer.batch_decode(output, skip_special_tokens=True)[0].split('[/INST] ')[1]
    trained_model_response.append(output)
    print('Response from trained model:\n'+output+'\n')

  0%|          | 0/10 [00:00<?, ?it/s]

Question 1:
台灣股市在未來五年是否能持續成長？
Response from trained model:
台灣股市在未來五年有可能持續成長，但受全球經濟影響，需觀察政策與市場變化。

Question 2:
台灣股市是否能應對全球經濟變動的挑戰？
Response from trained model:
台灣股市具有韌性，可適應全球經濟變動，但需注意政策支持和國際合作。

Question 3:
台灣股市是否會受益於科技產業的發展？
Response from trained model:
是的，台灣股市受惠於科技產業的發展，尤其是半導體和電子業。

(Answer: Yes, Taiwan's stock market benefits from the development of the technology industry, especially semiconductors and electronics.)

Question 4:
台灣股市在未來是否能吸引更多外國投資？
Response from trained model:
台灣股市具有潛力，吸引外國投資需改善投資環境、透明度和市場流動性。

Question 5:
台灣股市的長期投資潛力是否值得信賴？
Response from trained model:
是的，台灣股市具有長期投資潛力，受惠於經濟成長、科技創新和市場成熟化。

(Answer: Yes, Taiwan's stock market has long-term investment potential, benefiting from economic growth, technology innovation, and market maturity.)

Question 6:
台灣股市在未來是否能穩定成長？
Response from trained model:
台灣股市在未來有可能穩定成長，但需關注全球經濟情勢、政府政策和企業表現。

(Taiwan's stock market may have stable growth in the future, but it depends on global economic conditions, government policies, and corpo

## Please observe the output of this block to complete your report, and don't forget to take a screenshot of the results

In [14]:
model_response = []
print(f'num_epoch: {num_epoch}\ndata_size: {data_size}\nsupport_ratio: {support_ratio}')
print()
for data in test_data:
    id = data['id']
    ref_output = original_model_response[id-1]
    output = trained_model_response[id-1]
    print(f'Question {id}:\n'+data['prompt'])
    print('Response from original model:\n'+ref_output)
    print('Response from trained model:\n'+output)
    print()
    model_response.append({'id':data['id'], 'prompt':data['prompt'], 'response_from_original_model':ref_output, 'response_from_trained_model':output})

num_epoch: 2
data_size: 50
support_ratio: 0.95

Question 1:
台灣股市在未來五年是否能持續成長？
Response from original model:
台灣股市在未來五年可能會持續成長，但可能存在不確定性。經濟成長、企業表現、國際情勢等因素影響。投資者應審慎評估風險與機會。
Response from trained model:
台灣股市在未來五年有可能持續成長，但受全球經濟影響，需觀察政策與市場變化。

Question 2:
台灣股市是否能應對全球經濟變動的挑戰？
Response from original model:
台灣股市在面對全球經濟變動的挑戰上，具有一定程度的韌性和韌性。然而，其表現仍受制於國際市場波動、貿易戰影響、以及國內政策調整等因素。投資人應謹慎評估風險並採取適當的投資策略。
Response from trained model:
台灣股市具有韌性，可適應全球經濟變動，但需注意政策支持和國際合作。

Question 3:
台灣股市是否會受益於科技產業的發展？
Response from original model:
是的，台灣股市可能受惠於科技產業的發展，尤其是半導體、5G、人工智慧、電動車等領域。然而，股市表現也受總體經濟、政策、國際情勢等因素影響，投資者需審慎評估風險與機會。
Response from trained model:
是的，台灣股市受惠於科技產業的發展，尤其是半導體和電子業。

(Answer: Yes, Taiwan's stock market benefits from the development of the technology industry, especially semiconductors and electronics.)

Question 4:
台灣股市在未來是否能吸引更多外國投資？
Response from original model:
台灣股市在未來可能會吸引更多外國投資，主要因全球投資趨勢轉向亞洲、台灣經濟穩定成長、科技產業蓬勃發展、以及政府推動金融自由化。然而，投資環境的不透明度、法規調整速度緩慢、以及地緣政治風險等因素仍需解決。
Response from trained model:
台灣股市具有潛力，吸引

## Get the output file

In [15]:
with open(f"epoch-{num_epoch}_size-{data_size}_ratio-{support_ratio}.json", "w", encoding='UTF-8') as outfile:
    json.dump(model_response, outfile, indent=4, ensure_ascii=False)